In [ ]:
from pathlib import Path
import sys

# Empezar desde la carpeta actual donde se ejecuta el notebook
current = Path().resolve()

# Buscar hacia arriba hasta encontrar config/settings.yaml
for parent in current.parents:
    if (parent / "config" / "settings.yaml").exists():
        project_root = parent
        break
else:
    raise FileNotFoundError("No se encontró config/settings.yaml en la ruta actual ni en sus carpetas superiores.")

# Mostrar y añadir al sys.path
print("📍 project_root detectado:", project_root)

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print("✅ project_root añadido a sys.path")


In [ ]:
import os
import yaml

# Ruta al archivo de configuración
config_path = project_root / "config" / "settings.yaml"
print("📄 Archivo de configuración encontrado en:", config_path)

# Leer contenido del archivo
with open(config_path, "r") as f:
    raw_config = f.read()

# Expansión de variables de entorno si existen (como ${OPENAI_API_KEY})
expanded_config = os.path.expandvars(raw_config)

# Parsear YAML a diccionario
config_data = yaml.safe_load(expanded_config)

# Mostrar el contenido
print("\n📜 Contenido expandido del YAML:")
print(config_data)


In [ ]:
from providers.provider_manager import ProviderManager

# Inicializar el manejador de proveedores
manager = ProviderManager(str(config_path))

# Verificar el proveedor actual cargado desde settings.yaml
print("🔌 Proveedor activo:", manager.get_current_provider_name())

# Mostrar todos los proveedores disponibles
print("📦 Proveedores disponibles:", manager.list_available_providers())


In [ ]:
# Obtener el proveedor actual desde el manager
provider = manager.get_provider()

# Enviar un prompt de prueba
prompt = "Haz un resumen del archivo que acabo de leer"

# Obtener respuesta
respuesta = provider.generate_text(prompt)

# Mostrar resultado
print("🧠 Prompt enviado:", prompt)
print("\n💬 Respuesta del modelo:")
print(respuesta)


In [ ]:
from agent_modules.command_agent import CommandAgent
from agent_modules.command_agent.task_generator import TaskGenerator

# Subclase temporal para TaskGenerator que usa el proveedor dinámico
class DynamicTaskGenerator(TaskGenerator):
    def __init__(self, provider):
        self.provider = provider

    def generate(self, input_text):
        return self.provider.generate_text(input_text)

# Crear el generador con el proveedor dinámico actual
dynamic_generator = DynamicTaskGenerator(manager.get_provider())

# Crear el agente y reemplazar su generador
agent = CommandAgent()
agent.generator = dynamic_generator

# Probar el agente con una entrada real
entrada = "Convierte el contenido del informe en una lista de tareas"
salida = agent.act(entrada)

# Mostrar resultado
print("🧠 Entrada al agente:", entrada)
print("✅ Respuesta del agente:", salida)
